# Prepare session

In [ ]:
import boto3
import sagemaker
from sagemaker import get_execution_role
from sagemaker.local import LocalSession
import s3fs
import subprocess
from sagemaker.s3 import S3Downloader, S3Uploader
from pathlib import Path
import json

image_name = "sagemaker-test"
ecr_namespace = image_name + "/"
default_bucket = "prod-test"
default_uri = "s3://" + default_bucket
atf_s3_uri = default_uri + "/sagemaker"

role = get_execution_role()
account_id = role.split(":")[4]
boto_session = boto3.Session()
region = boto_session.region_name
bucket = default_bucket

sagemaker_session = sagemaker.Session(
    boto_session=boto_session,
    default_bucket=default_bucket
)
    
s3_helper = s3fs.S3FileSystem()
data_location_uri = default_uri + "/training_data/full"

print(account_id)
print(region)
print(role)
print(sagemaker_session)
print(default_uri)
print(atf_s3_uri)
print(data_location_uri)

# Dev in real

## Build and push image

In [ ]:
! cd container && bash build_image.sh $image_name

In [ ]:
! cd container && bash push_image.sh $image_name

In [ ]:
ecr_client = boto3.client('ecr')
response = ecr_client.describe_images(
    repositoryName=image_name,
    imageIds=[{'imageTag': 'latest'}],
)
str(response["imageDetails"][0]["imagePushedAt"])

## Define parameters

In [ ]:
opt_ml_dir = "/opt/ml/processing"
execution_id = "exp-real-sm"
image_uri = f"{account_id}.dkr.ecr.{region}.amazonaws.com/{image_name}:latest"
print(image_uri)

In [6]:
processing_instance_count = 1
processing_instance_type = "ml.m5.2xlarge"
training_instance_type = "ml.m5.2xlarge"

## Prepare model

In [ ]:
train_data_uri = atf_s3_uri + f"/prepared_data/{execution_id}/train"
test_data_uri = atf_s3_uri + f"/prepared_data/{execution_id}/test"
! aws s3 ls $train_data_uri/
! aws s3 ls $test_data_uri/

In [ ]:
model_data_s3_uri = atf_s3_uri + "/model/sagemaker-test-2021-11-27-07-49-22-086/output/model.tar.gz"
model_data_s3_uri

In [11]:
from sagemaker.model import Model
model_name = "exp-real-sm-model"
model = Model(
    name=model_name,
    image_uri=image_uri,
    model_data=model_data_s3_uri,
    role=role
)

## Deploy

In [ ]:
# this function returns None
endpoint_name = "exp-real-sm-01"
print("EndpointName={}".format(endpoint_name))

predictor = model.deploy(
    initial_instance_count=1,
    instance_type=training_instance_type,
    endpoint_name=endpoint_name,
)

In [13]:
from sagemaker.predictor import Predictor
from sagemaker.predictor import CSVSerializer
predictor = Predictor(
    endpoint_name=endpoint_name,
    sagemaker_session=sagemaker_session,
    serializer=CSVSerializer()
)

In [ ]:
endpoint_name = predictor.endpoint_name
runtime = boto3.Session().client("runtime.sagemaker")
print(endpoint_name)
print(runtime)

## Test endpoint

In [ ]:
import pandas as pd
s3 = boto3.client('s3')
obj = s3.get_object(Bucket=default_bucket, Key=f'sagemaker/prepared_data/{execution_id}/train/train.csv')
train_df = pd.read_csv(obj['Body']) # 'Body' is a key word
train_df.head()

In [ ]:
test_data = train_df.drop(train_df.columns[[0]], axis=1)
test_data.head()

### Test endpoint using predict function

In [ ]:
def format_results(results):
    return list(map(float, results.split('\n')[:-1]))
results = predictor.predict(test_data.values).decode('utf-8')
format_results(results)

### Test endpoint using invoke_endpoint function

In [ ]:
response = runtime.invoke_endpoint(
    EndpointName=endpoint_name,
    Body=CSVSerializer().serialize(test_data.values),
    ContentType='text/csv',
)
format_results(response['Body'].read().decode())

In [ ]:
body = CSVSerializer().serialize(test_data.values[:2])
body

In [ ]:
response = runtime.invoke_endpoint(
    EndpointName=endpoint_name,
    Body=body,
    ContentType='text/csv',
)
format_results(response['Body'].read().decode())

### Test endpoint using invoke_endpoint command line

In [ ]:
test_data_str = CSVSerializer().serialize(test_data.values)
payload_file = "./payload"
with open(payload_file, "w") as f:
    f.write(test_data_str)
! aws sagemaker-runtime invoke-endpoint --endpoint-name $endpoint_name --body fileb://./payload --content-type text/csv outfile.txt && cat outfile.txt

In [ ]:
# print out logs from Cloud Watch
logs = boto3.client("logs")

log_res = logs.describe_log_streams(
    logGroupName=f"/aws/sagemaker/Endpoints/{endpoint_name}"
)

for log_stream in log_res["logStreams"]:
    # get one log event
    log_event = logs.get_log_events(
        logGroupName=f"/aws/sagemaker/Endpoints/{endpoint_name}", logStreamName=log_stream["logStreamName"]
    )

    # print out messages from the log event
    for ev in log_event["events"]:
        for k, v in ev.items():
            if k == "message":
                print(v)

## Build lambda container

### Build real lambda image

In [ ]:
lambda_real = "lambda-test"
! cd container && bash build_image.sh $lambda_real Dockerfile.lambda.ecr

In [ ]:
! cd container && bash push_image.sh $lambda_real

Manually create/update lambda function using the above pushed image.

Config:
- Memory: 512MB
- Timeout: 20s

Make sure lambda role allows:
- All sagemaker resources
    - "sagemaker:InvokeEndpoint"
    - "sagemaker:GetRecord"
    - "sagemaker:PutRecord"
- S3 bucket: prod-test*
    - List, Read

Use payload file's content to test the lambda function

Create trigger API Gateway

In [ ]:
# Create your own payload file
payload = "./payload"

In [ ]:
# Test API Gateway
api_gateway_url = "https://xc8mtwmk9e.execute-api.ap-southeast-1.amazonaws.com/deploy/lambda-test"
! curl -XPOST $api_gateway_url --data-binary @payload

## Clean

In [32]:
predictor.delete_endpoint()

In [33]:
model.delete_model()

In [34]:
! rm payload && rm outfile.txt && rm test.wav

Delete lambda function and API Gateway